<a href="https://colab.research.google.com/github/jpcloud2021/pyspark-exm/blob/main/json_db_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 67kB/s 
     |████████████████████████████████| 204kB 18.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=55387d468d53af12a4293ad6ae41eadf9ed38568508a2d794eeaefa7a03ecbcd
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [14]:
#test_emp.json
data=[{
   "data": {
       "emp_id": "12345",
       "emp_name": "Mohan",
       "awards": [
           {
               "award_type": "Internal",
               "award_name": "Best_emp_of_the_year",
               "year": "2000"
           },
           {
               "award_type": "External",
               "award_name": "Best_presenter",
               "year": "2001"
           }
       ]
   }
}]

#main.py
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StringType, StructType
 
schema = StructType() \
             .add("data", StructType() \
                   .add("emp_id", StringType(),True) \
                   .add("emp_name", StringType(), True) \
                   .add("awards", ArrayType(StructType()                     
                       .add("award_type", StringType(), True) \
                       .add("award_name", StringType(), True))) \
               , True)
 


In [15]:
spark= SparkSession.builder\
.appName("MyJsonProcess")\
.master("local[*]")\
.getOrCreate()

spark
 
df1 = spark.createDataFrame(data=data,schema=schema)
df1.show(truncate=False)
df1.printSchema()

+------------------------------------------------------------------------------+
|data                                                                          |
+------------------------------------------------------------------------------+
|{12345, Mohan, [{Internal, Best_emp_of_the_year}, {External, Best_presenter}]}|
+------------------------------------------------------------------------------+

root
 |-- data: struct (nullable = true)
 |    |-- emp_id: string (nullable = true)
 |    |-- emp_name: string (nullable = true)
 |    |-- awards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- award_type: string (nullable = true)
 |    |    |    |-- award_name: string (nullable = true)

